# Uso de algoritmos genéticos para búsqueda de hiper parámetros

Fuente: [*Tune Your Scikit-learn Model Using Evolutionary Algorithms*](https://towardsdatascience.com/tune-your-scikit-learn-model-using-evolutionary-algorithms-30538248ac16)

Autor: Rodrigo Arenas (towards data science)

Para instalar el paquete

`$ pip install sklearn-genetic-opt`


### Ejemplo con un RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
import seaborn as sns

data = load_digits() 

n_samples = len(data.images)
X = data.images.reshape((n_samples, -1))
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7)

clf = RandomForestClassifier()

### Uso de `sklearn-genetic-opt`

Elementos importantes:
- `param_grid`
- `GASearchCV`


In [ ]:
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Continuous, Categorical, Integer
from sklearn_genetic.plots import plot_fitness_evolution, plot_search_space
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

param_grid = {'min_weight_fraction_leaf': Continuous(0.01, 0.5, distribution='log-uniform'),
              'bootstrap': Categorical([True, False]),
              'max_depth': Integer(2, 30), 
              'max_leaf_nodes': Integer(2, 35), 
              'n_estimators': Integer(100, 300)}

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=clf,
                               cv=cv,
                               scoring='accuracy',
                               population_size=10,
                               generations=10,
                               tournament_size=3,
                               elitism=True,
                               crossover_probability=0.8,
                               mutation_probability=0.1,
                               param_grid=param_grid,
                               criteria='max',
                               algorithm='eaMuPlusLambda',
                               n_jobs=-1,
                               verbose=True,
                               keep_top_k=4)

### Inicio del ciclo

In [ ]:
evolved_estimator.fit(X_train,y_train)


### Una predicción

In [ ]:
y_predicy_ga = evolved_estimator.predict(X_test)
accuracy_score(y_test,y_predicy_ga)

### Mejores parámetros

In [ ]:
evolved_estimator.best_params_

In [ ]:
import seaborn as sns
plot_fitness_evolution(evolved_estimator)
plt.show()

In [ ]:
print(evolved_estimator.logbook)

In [ ]:
evolved_estimator.hof